Imports

In [1]:
import numpy as np
import pandas as pd

from keras.utils import to_categorical
from keras.layers import Dense, Activation, Conv2D, Dropout
from keras.models import Sequential

Data Pre-processing (getting from dataset_builder.py) will go here

In [5]:
df = pd.read_csv('dataset.csv')

print(df.shape)

df = df.reshape(60,64,64)

print(df.info)

(3775, 64)


Generator Model

In [ ]:
def make_generator():
    model = Sequential()

Discriminator Model

In [ ]:
def make_discriminator():
    model = Sequential()
    